In [1]:
import numpy as np
import csv
import pandas as pd

In [2]:
# # dedicated runtime
# appRuntime = np.load('../dedicated_runtime/app_dedicated_rt.npy').item()

In [3]:
app_at_gpu0 = np.load('../../02_dedicate/homedesktop-gpu0.npy').item()
# print app_at_gpu0

app_at_gpu1 = np.load('../../02_dedicate/homedesktop-gpu1.npy').item()
# print app_at_gpu1

appRuntime = {}
for app, gpu0_rt in app_at_gpu0.iteritems():
    gpu1_rt = app_at_gpu1[app]
    appRuntime[app] = min(gpu0_rt, gpu1_rt)

In [4]:
# appRuntime

In [5]:
def read_timing_df(curfile):
    """read timing traces in csv file, offset the trace with the starting pos, return the dataframe"""
    df_current = pd.read_csv(curfile)
    offset = df_current['start'].min()  # find the starting timing, and offset it
    df_current['start'] = df_current['start'].apply(lambda x: x - offset)
    df_current['end'] = df_current['end'].apply(lambda x: x - offset)
    return df_current

def compare_with_dedicatedruntime(df, appRuntime):
    """
    read the timing traces in dataframe (after read_timing_df()).
    compute the speedup over the dedicated runtime
    return speedup list
    """
    dedicated_total,  corun_total = 0.0, 0.0
    speedup_list = []

    # columns => ['jobID', 'appName', 'start', 'end', 'duration (s)']
    for index, row in df.iterrows():
        app_name, app_endT = row['appName'], row['end']
        app_rt_dedicate = float(appRuntime[app_name])  # dedicated runtime
        app_endT = float(app_endT)
        if corun_total < app_endT: corun_total = app_endT
        dedicated_total += app_rt_dedicate
        speedup = dedicated_total / corun_total
        speedup_list.append(speedup)   
    return speedup_list

In [6]:
len(appRuntime)

75

In [7]:
appRuntime

{'cudasdk_BlackScholes': 1.9693429470062256,
 'cudasdk_FDTD3d': 9.9745099544525146,
 'cudasdk_MCEstimatePiInlineP': 0.26801586151123047,
 'cudasdk_MCEstimatePiInlineQ': 0.24531793594360352,
 'cudasdk_MCEstimatePiP': 0.28968906402587891,
 'cudasdk_MCEstimatePiQ': 0.27803206443786621,
 'cudasdk_MCSingleAsianOptionP': 0.2678530216217041,
 'cudasdk_SobolQRNG': 0.31908392906188965,
 'cudasdk_batchCUBLAS': 0.37936806678771973,
 'cudasdk_binomialOptions': 22.217447996139526,
 'cudasdk_boxFilterNPP': 0.25980210304260254,
 'cudasdk_c++11Cuda': 0.84449291229248047,
 'cudasdk_concurrentKernels': 0.24102401733398438,
 'cudasdk_convolutionFFT2D': 3.0999901294708252,
 'cudasdk_convolutionSeparable': 1.5115499496459961,
 'cudasdk_convolutionTexture': 0.90807485580444336,
 'cudasdk_dct8x8': 0.4726259708404541,
 'cudasdk_dwtHaar1D': 0.23399615287780762,
 'cudasdk_dxtc': 0.26365184783935547,
 'cudasdk_eigenvalues': 2.1058349609375,
 'cudasdk_fastWalshTransform': 2.7304399013519287,
 'cudasdk_interval': 

In [8]:
def compute_stp(inputcsv):
    """Read app status table in csv format, compute the ANTT"""
    
    df = read_timing_df(inputcsv)
    
    #
    # 1) sort the dataframe by 'end'
    #
    sorted_df = df.sort_values(by=['end'])
    
    
    #
    # 2) compute the stp for each app
    # 
    stp_list = []
    for idx, row in sorted_df.iterrows():
        appName = row['appName']
        T_corun = float(row['duration (s)'])
        T_dedicate = float(appRuntime[appName])

        tmp =  T_dedicate / T_corun
        stp_list.append(tmp)

    #print stp_list
    
    #
    # 3) in stp list, add current value with prev
    # 
    stp = [0 for i in xrange(len(stp_list))]
    for i, v in enumerate(stp_list):
        if i == 0:
            stp[0] = v
        else:
            stp[i] = stp[i-1] + v
    
    return stp
    

### read traces

In [9]:
# inputcsv = "../fcfs_csv/fcfs_run8_s1.csv"
# stp = compute_stp(inputcsv)
# print stp

# mydf = pd.DataFrame(stp)
# mydf.to_csv('stp_fcfs_max8.csv', index=False, header=False)

In [10]:
prefix = "ll"
for sid in [1,2,3]:
    for cid in [2,4,6,8]:
        inputcsv = prefix + "-s" + str(sid) + "-c" + str(cid) + ".csv"
        antt = compute_stp(inputcsv)
        print inputcsv, antt[-1]
    print "\n"

ll-s1-c2.csv 43.294096957
ll-s1-c4.csv 25.9461163075
ll-s1-c6.csv 13.0208252945
ll-s1-c8.csv 11.3686947955


ll-s2-c2.csv 29.7406122005
ll-s2-c4.csv 19.1059474497
ll-s2-c6.csv 13.6690911357
ll-s2-c8.csv 11.8805757976


ll-s3-c2.csv 41.0530151517
ll-s3-c4.csv 18.4197193782
ll-s3-c6.csv 15.0715238904
ll-s3-c8.csv 11.1728892932




In [11]:
prefix = "rr"
for sid in [1,2,3]:
    for cid in [2,4,6,8]:
        inputcsv = prefix + "-s" + str(sid) + "-c" + str(cid) + ".csv"
        antt = compute_stp(inputcsv)
        print inputcsv, antt[-1]
    print "\n"

rr-s1-c2.csv 39.5783664545
rr-s1-c4.csv 22.0660895839
rr-s1-c6.csv 15.7546082523
rr-s1-c8.csv 11.5475186766


rr-s2-c2.csv 37.5774212307
rr-s2-c4.csv 20.8530368451
rr-s2-c6.csv 14.0342776989
rr-s2-c8.csv 12.3664005075


rr-s3-c2.csv 36.4797955137
rr-s3-c4.csv 28.8815689666
rr-s3-c6.csv 19.7537679568
rr-s3-c8.csv 14.5165638052




In [12]:
prefix = "xrbfeatAll"
for sid in [1,2,3]:
    for cid in [2,4,6,8]:
        inputcsv = prefix + "-s" + str(sid) + "-c" + str(cid) + ".csv"
        antt = compute_stp(inputcsv)
        print inputcsv, antt[-1]
    print "\n"

xrbfeatAll-s1-c2.csv 52.3424662771
xrbfeatAll-s1-c4.csv 26.7862065505
xrbfeatAll-s1-c6.csv 16.3634967964
xrbfeatAll-s1-c8.csv 12.8437752527


xrbfeatAll-s2-c2.csv 49.7191162431
xrbfeatAll-s2-c4.csv 28.5964289773
xrbfeatAll-s2-c6.csv 16.5690299207
xrbfeatAll-s2-c8.csv 12.0506863939


xrbfeatAll-s3-c2.csv 57.0411406514
xrbfeatAll-s3-c4.csv 30.8259890337
xrbfeatAll-s3-c6.csv 18.2643951246
xrbfeatAll-s3-c8.csv 14.307913732




In [13]:
prefix = "xrbfeat9"
for sid in [1,2,3]:
    for cid in [2,4,6,8]:
        inputcsv = prefix + "-s" + str(sid) + "-c" + str(cid) + ".csv"
        antt = compute_stp(inputcsv)
        print inputcsv, antt[-1]
    print "\n"

xrbfeat9-s1-c2.csv 51.8525917031
xrbfeat9-s1-c4.csv 32.3101594152
xrbfeat9-s1-c6.csv 17.6011874403
xrbfeat9-s1-c8.csv 13.3793183406


xrbfeat9-s2-c2.csv 45.9316601078
xrbfeat9-s2-c4.csv 34.8243324055
xrbfeat9-s2-c6.csv 19.3183406771
xrbfeat9-s2-c8.csv 12.9888398019


xrbfeat9-s3-c2.csv 58.6307011915
xrbfeat9-s3-c4.csv 32.4119374777
xrbfeat9-s3-c6.csv 19.9745845734
xrbfeat9-s3-c8.csv 13.2302454477


